In [126]:
# Notebook to preprocess the datasets and create new datasets required for the analysis.
# import required packages
import numpy as np
import pandas as pd

drivers=pd.read_csv('.//data//drivers.csv')
champresults=pd.read_csv('.//data//driver_standings.csv')
qualiresults=pd.read_csv('.//data//qualifying.csv')
racedata=pd.read_csv('.//data//races.csv')
raceresults=pd.read_csv('.//data//results.csv')

Build array with the raceId of the first and last race each year.

In [127]:
racedataPerYear=racedata.groupby('year')
listracesperyear=[racedataPerYear.get_group(x) for x in racedataPerYear.groups]
raceIdyearSplitter=np.array([[x['year'].min(), x['raceId'].min(), x['raceId'].max()] for x in listracesperyear ])
#Convert the previous list to a dictionary for easier access in later steps
raceIdyearDictio={'{}'.format(x[0]):x[1:] for i, x in enumerate(raceIdyearSplitter)}

Select a year from which we will start considering the results and then build an array with each driver in each year. If a driver competed several years, they will appear several times since each competed season will be a different data point.

In [128]:
minYearToConsider=1990
i, = np.where( raceIdyearSplitter[:,0] == 1990)
data_array=[]
for j in range(int(i),len(raceIdyearSplitter[:,0])-2,1): #I exclude the year 2023 due to being incomplete.
    for ind, row in champresults[champresults['raceId']==raceIdyearSplitter[j,2]][['driverId','position']].iterrows():
        data_array.append([raceIdyearSplitter[j,0], row['driverId'],row['position']])

Convert the array to a Pandas DataFrame. This is the basic Data Frame we will use.
Now, we will append columns describing the season of each driver in each year: race results, quali results.

In [129]:
driverData=pd.DataFrame(data_array,columns=['Year','Driver','Championship Position'])

In [130]:
driverData.head()

,Year,Driver,Championship Position
0,1990,65,35
1,1990,152,39
2,1990,88,12
3,1990,95,5
4,1990,77,4


Generate the column with the race results for each driver in each year.
Consider the finishing position without considering whether they completed the race distance or not. This could generate problems with outliers where they finish high enough to score points but didn't complete enough laps

In [131]:
driverData=driverData.assign(RaceResults=None)
for i, dri in driverData.iterrows():
    resdri=raceresults[(raceresults['raceId']>=raceIdyearDictio[str(dri['Year'])][0])*(raceresults['raceId']<=raceIdyearDictio[str(dri['Year'])][1])*(raceresults['driverId']==dri['Driver'])]['positionOrder'].reset_index(drop=True).to_numpy()
    driverData.at[i,'RaceResults']=resdri.astype(float)

In [132]:
driverData.head(5)

,Year,Driver,Championship Position,RaceResults
0,1990,65,35,"[12.0, 17.0]"
1,1990,152,39,"[33.0, 35.0, 35.0, 35.0, 35.0, 35.0, 35.0, 35...."
2,1990,88,12,"[16.0, 21.0, 21.0, 21.0, 12.0, 23.0, 7.0, 6.0,..."
3,1990,95,5,"[17.0, 4.0, 15.0, 9.0, 3.0, 2.0, 18.0, 15.0, 1..."
4,1990,77,4,"[18.0, 2.0, 2.0, 3.0, 4.0, 3.0, 5.0, 14.0, 3.0..."


In [133]:

#For each driver in each season, get a vector with all their race and qualy positions.
#The champresults at each final race seems to contain all the drivers that took part in that year.
#THe results at initial races contain only the ones that scored points at the beginning.